# CMIP6 Evaporation (Evap) Calculation

## Overview

This notebook calculates evaporation (evapotranspiration) from CMIP6 climate data. Evaporation can be calculated using various methods depending on available variables.

## Input Variables

- **tasmax**: Daily maximum temperature (°C) - required
- **tasmin**: Daily minimum temperature (°C) - required
- **rsds**: Surface downwelling shortwave radiation (W/m²) - required
- **hurs**: Relative humidity (%) - optional but recommended
- **pr**: Precipitation (mm) - optional

## Output

- **evap**: Evaporation/Evapotranspiration (mm/day)

## Calculation Methods

### Method 1: Simplified Penman-Monteith (Recommended)

Uses temperature, radiation, and humidity to estimate evapotranspiration:

```
ET = (Δ × Rn + γ × (900/(T+273)) × u2 × (es - ea)) / (Δ + γ × (1 + 0.34 × u2))
```

Where:
- `Δ` = slope of vapor pressure curve
- `Rn` = net radiation (MJ/m²/day)
- `γ` = psychrometric constant
- `T` = mean temperature (°C)
- `u2` = wind speed at 2m (m/s) - estimated if not available
- `es` = saturation vapor pressure (hPa)
- `ea` = actual vapor pressure (hPa)

### Method 2: Temperature-based (Hargreaves-Samani)

Simpler method using only temperature and radiation:

```
ET = 0.0023 × Ra × (Tmean + 17.8) × √(Tmax - Tmin)
```

Where:
- `Ra` = extraterrestrial radiation (MJ/m²/day)
- `Tmean` = mean temperature (°C)
- `Tmax` = maximum temperature (°C)
- `Tmin` = minimum temperature (°C)

## Usage

1. Set configuration parameters (Model, Scenario, coordinates)
2. Extract required climate variables from NetCDF files
3. Calculate evaporation using selected method
4. Save results to CSV file

## Section 1: Imports and Configuration

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import glob
import os
import time
import re
import math
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

print("Libraries imported successfully")

In [ ]:
# Configuration
CMIP6_BASE_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\CMIP6"
OUTPUT_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka_South_16_226042"  # Output directory
COORD_TOLERANCE = 0.01  # degrees (approximately 1.1 km)

# Model and Scenario
MODEL = "ACCESS CM2"  # e.g., "ACCESS CM2"
SCENARIO = "SSP585"   # e.g., "SSP245" or "SSP585"

# Coordinates
LATITUDE = -31.75   # Target latitude in decimal degrees (-90 to 90)
LONGITUDE = 117.5999984741211  # Target longitude in decimal degrees (-180 to 180)

# Calculation method: 'penman' or 'hargreaves'
EVAP_METHOD = 'hargreaves'  # Default to Hargreaves (simpler, fewer variables needed)

# Variables required for evaporation calculation
REQUIRED_VARIABLES = ['tasmax', 'tasmin', 'rsds']  # Minimum required
OPTIONAL_VARIABLES = ['hurs', 'pr']  # Optional but improve accuracy

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Configuration loaded:")
print(f"  - CMIP6 Base Directory: {CMIP6_BASE_DIR}")
print(f"  - Output Directory: {OUTPUT_DIR}")
print(f"  - Model: {MODEL}")
print(f"  - Scenario: {SCENARIO}")
print(f"  - Coordinates: ({LATITUDE:.6f}, {LONGITUDE:.6f})")
print(f"  - Calculation Method: {EVAP_METHOD}")
print(f"  - Required Variables: {', '.join(REQUIRED_VARIABLES)}")
print(f"  - Optional Variables: {', '.join(OPTIONAL_VARIABLES)}")

## Section 2: NetCDF Data Extraction Function

In [ ]:
def extract_daily_data_from_netcdf(netcdf_dir, variable, target_lat, target_lon, tolerance=0.01):
    """
    Extract daily time series data for a specific coordinate from NetCDF files.
    
    Parameters:
    -----------
    netcdf_dir : str
        Directory containing NetCDF files for the variable
    variable : str
        Variable name (tasmax, tasmin, rsds, hurs, pr)
    target_lat : float
        Target latitude
    target_lon : float
        Target longitude
    tolerance : float
        Coordinate matching tolerance in degrees
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: date, value
    """
    start_time = time.time()
    
    # Find all NetCDF files in the directory
    nc_files = sorted(glob.glob(os.path.join(netcdf_dir, f"*{variable}*.nc")))
    
    # Pattern 2: Files in subdirectories named {variable}_*
    if len(nc_files) == 0:
        var_subdirs = glob.glob(os.path.join(netcdf_dir, f"{variable}_*"))
        for var_subdir in var_subdirs:
            if os.path.isdir(var_subdir):
                found_files = sorted(glob.glob(os.path.join(var_subdir, "*.nc")))
                if found_files:
                    nc_files.extend(found_files)
                    print(f"  Found files in subdirectory: {os.path.basename(var_subdir)}/")
                    break
    
    # For rsds, also check rad_* folders
    if len(nc_files) == 0 and variable == 'rsds':
        rad_subdirs = glob.glob(os.path.join(netcdf_dir, "rad_*"))
        for rad_subdir in rad_subdirs:
            if os.path.isdir(rad_subdir):
                found_files = sorted(glob.glob(os.path.join(rad_subdir, "*rsds*.nc")))
                if found_files:
                    nc_files.extend(found_files)
                    print(f"  Found files in subdirectory: {os.path.basename(rad_subdir)}/")
                    break
    
    if len(nc_files) == 0:
        print(f"  ERROR: No NetCDF files found in {netcdf_dir}")
        return None
    
    print(f"  Found {len(nc_files)} NetCDF files")
    
    # Cache coordinate information from first file
    lat_name = None
    lon_name = None
    time_name = None
    lat_idx = None
    lon_idx = None
    var_name = None
    
    # List to store daily data
    all_data = []
    
    # Process first file to get coordinate structure
    if len(nc_files) > 0:
        try:
            ds_sample = xr.open_dataset(nc_files[0], decode_times=False)
            
            # Get variable name
            for v in ds_sample.data_vars:
                if variable in v.lower() or v.lower() in variable.lower():
                    var_name = v
                    break
            
            if var_name is None:
                possible_names = [variable, variable.upper(), f'{variable}_day']
                if variable == 'rsds':
                    possible_names.extend(['rad', 'RAD', 'rad_day'])
                for name in possible_names:
                    if name in ds_sample.data_vars:
                        var_name = name
                        break
            
            # Get coordinate names
            for coord in ds_sample.coords:
                coord_lower = coord.lower()
                if 'lat' in coord_lower:
                    lat_name = coord
                elif 'lon' in coord_lower:
                    lon_name = coord
                elif 'time' in coord_lower:
                    time_name = coord
            
            if lat_name and lon_name:
                # Find nearest grid point
                lat_idx = np.abs(ds_sample[lat_name].values - target_lat).argmin()
                lon_idx = np.abs(ds_sample[lon_name].values - target_lon).argmin()
                
                actual_lat = float(ds_sample[lat_name].values[lat_idx])
                actual_lon = float(ds_sample[lon_name].values[lon_idx])
                
                # Check if within tolerance
                if abs(actual_lat - target_lat) > tolerance or abs(actual_lon - target_lon) > tolerance:
                    print(f"  Warning: Nearest point ({actual_lat:.4f}, {actual_lon:.4f}) is outside tolerance")
                else:
                    print(f"  Using grid point: ({actual_lat:.4f}, {actual_lon:.4f})")
            
            ds_sample.close()
            
        except Exception as e:
            print(f"  Warning: Could not read sample file: {e}")
    
    if var_name is None or lat_idx is None or lon_idx is None:
        print(f"  ERROR: Could not determine coordinate structure")
        return None
    
    # Process all files with progress bar
    print(f"  Processing files...")
    for nc_file in tqdm(nc_files, desc=f"  {variable}", unit="file"):
        try:
            ds = xr.open_dataset(nc_file, decode_times=False)
            
            # Extract data using cached indices
            data = ds[var_name].isel({lat_name: lat_idx, lon_name: lon_idx})
            
            # Convert to numpy array
            values = data.values
            if values.ndim > 1:
                values = values.flatten()
            
            # Get time values
            time_values = None
            
            # Method 1: Try to use time coordinate from NetCDF file
            if time_name and time_name in ds.coords:
                try:
                    time_coord = ds[time_name]
                    if len(time_coord) == len(values):
                        try:
                            time_decoded = xr.decode_cf(ds[[time_name]])[time_name]
                            time_values = pd.to_datetime(time_decoded.values)
                        except:
                            if hasattr(time_coord, 'units') and 'days since' in time_coord.units.lower():
                                base_date_str = time_coord.units.split('since')[1].strip().split()[0]
                                base_date = pd.to_datetime(base_date_str)
                                time_values = base_date + pd.to_timedelta(time_coord.values, unit='D')
                except Exception as e:
                    pass
            
            # Method 2: Extract year from filename
            if time_values is None:
                year = None
                filename = os.path.basename(nc_file)
                all_years = re.findall(r'\d{4}', filename)
                for year_str in all_years:
                    year_candidate = int(year_str)
                    if 2000 <= year_candidate <= 2100:
                        year = year_candidate
                        break
                
                if year:
                    time_values = pd.date_range(start=f'{year}-01-01', periods=len(values), freq='D')
                else:
                    time_values = pd.date_range(start='2035-01-01', periods=len(values), freq='D')
            
            # Ensure correct number of dates
            if len(time_values) != len(values):
                if len(time_values) > len(values):
                    time_values = time_values[:len(values)]
            
            # Create DataFrame for this file
            if len(values) > 0:
                df_file = pd.DataFrame({
                    'date': time_values[:len(values)],
                    'value': values
                })
                all_data.append(df_file)
            
            ds.close()
            
        except Exception as e:
            tqdm.write(f"    Error processing {os.path.basename(nc_file)}: {e}")
            continue
    
    if len(all_data) == 0:
        print(f"  ERROR: No data extracted")
        return None
    
    # Combine all data
    print(f"  Combining data from {len(all_data)} files...")
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Sort by date
    combined_df = combined_df.sort_values('date').reset_index(drop=True)
    
    # Remove duplicate dates (keep first occurrence)
    combined_df = combined_df.drop_duplicates(subset='date', keep='first')
    
    elapsed_time = time.time() - start_time
    print(f"  ✓ Extracted {len(combined_df):,} daily records in {elapsed_time:.1f} seconds")
    print(f"  Date range: {combined_df['date'].min()} to {combined_df['date'].max()}")
    
    return combined_df

## Section 3: Evaporation Calculation Functions

In [ ]:
def calculate_evaporation_hargreaves(tasmax_df, tasmin_df, rsds_df, latitude):
    """
    Calculate evapotranspiration using Hargreaves-Samani method.
    
    Parameters:
    -----------
    tasmax_df : pd.DataFrame
        DataFrame with date and value (maximum temperature °C) columns
    tasmin_df : pd.DataFrame
        DataFrame with date and value (minimum temperature °C) columns
    rsds_df : pd.DataFrame
        DataFrame with date and value (solar radiation W/m²) columns
    latitude : float
        Latitude in decimal degrees
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with date and value (evaporation mm/day) columns
    """
    # Merge all dataframes
    merged = tasmax_df.merge(tasmin_df, on='date', suffixes=('_max', '_min'))
    merged = merged.merge(rsds_df, on='date')
    merged = merged.rename(columns={'value': 'rsds'})
    
    # Calculate mean temperature
    merged['tmean'] = (merged['value_max'] + merged['value_min']) / 2.0
    merged['trange'] = merged['value_max'] - merged['value_min']
    
    # Convert rsds from W/m² to MJ/m²/day
    merged['rsds_mj'] = merged['rsds'] * 0.0864  # W/m² to MJ/m²/day
    
    # Calculate day of year
    merged['doy'] = merged['date'].dt.dayofyear
    
    # Calculate extraterrestrial radiation (Ra) in MJ/m²/day
    # Simplified formula for daily Ra
    lat_rad = math.radians(latitude)
    
    def calculate_ra(doy):
        # Solar declination (radians)
        delta = 0.409 * math.sin(2 * math.pi * doy / 365 - 1.39)
        # Sunset hour angle (radians)
        ws = math.acos(-math.tan(lat_rad) * math.tan(delta))
        # Extraterrestrial radiation (MJ/m²/day)
        dr = 1 + 0.033 * math.cos(2 * math.pi * doy / 365)
        ra = 37.6 * dr * (ws * math.sin(lat_rad) * math.sin(delta) + math.cos(lat_rad) * math.cos(delta) * math.sin(ws))
        return ra
    
    merged['ra'] = merged['doy'].apply(calculate_ra)
    
    # Hargreaves-Samani formula: ET = 0.0023 × Ra × (Tmean + 17.8) × √(Tmax - Tmin)
    # Using actual solar radiation instead of Ra for better accuracy
    merged['evap'] = 0.0023 * merged['rsds_mj'] * (merged['tmean'] + 17.8) * np.sqrt(merged['trange'])
    
    # Ensure non-negative values
    merged['evap'] = merged['evap'].clip(lower=0.0)
    
    # Return DataFrame with date and evap columns
    evap_df = merged[['date', 'evap']].copy()
    evap_df = evap_df.rename(columns={'evap': 'value'})
    
    return evap_df

## Section 4: Main Processing

In [ ]:
# Construct data directory path
data_dir = os.path.join(CMIP6_BASE_DIR, f"{MODEL} {SCENARIO}")

if not os.path.exists(data_dir):
    raise ValueError(f"Data directory not found: {data_dir}")

print("="*70)
print(f"Processing Coordinate: ({LATITUDE:.6f}, {LONGITUDE:.6f})")
print(f"Model: {MODEL}, Scenario: {SCENARIO}")
print("="*70)
print(f"\nData directory: {data_dir}\n")

# Extract data for all required variables
extracted_data = {}

for variable in REQUIRED_VARIABLES:
    print(f"\n{'='*70}")
    print(f"Processing variable: {variable}")
    print(f"{'='*70}")
    
    df = extract_daily_data_from_netcdf(
        data_dir, 
        variable, 
        LATITUDE, 
        LONGITUDE, 
        tolerance=COORD_TOLERANCE
    )
    
    if df is not None and len(df) > 0:
        extracted_data[variable] = df
        print(f"  [OK] Extracted {len(df):,} records for {variable}")
    else:
        print(f"  [ERROR] Failed to extract data for {variable}")

# Try to extract optional variables
for variable in OPTIONAL_VARIABLES:
    print(f"\n{'='*70}")
    print(f"Processing optional variable: {variable}")
    print(f"{'='*70}")
    
    df = extract_daily_data_from_netcdf(
        data_dir, 
        variable, 
        LATITUDE, 
        LONGITUDE, 
        tolerance=COORD_TOLERANCE
    )
    
    if df is not None and len(df) > 0:
        extracted_data[variable] = df
        print(f"  [OK] Extracted {len(df):,} records for {variable}")
    else:
        print(f"  [WARNING] Could not extract data for optional variable {variable}")

# Check if all required variables are available
missing_vars = [v for v in REQUIRED_VARIABLES if v not in extracted_data]

if missing_vars:
    raise ValueError(f"Missing required variables: {missing_vars}")

print(f"\n{'='*70}")
print(f"Calculating Evaporation using {EVAP_METHOD} method...")
print(f"{'='*70}")

# Calculate evaporation based on selected method
if EVAP_METHOD == 'hargreaves':
    evap_df = calculate_evaporation_hargreaves(
        extracted_data['tasmax'],
        extracted_data['tasmin'],
        extracted_data['rsds'],
        LATITUDE
    )
else:
    raise ValueError(f"Unknown evaporation method: {EVAP_METHOD}")

print(f"  [OK] Calculated evaporation for {len(evap_df):,} days")
print(f"  Date range: {evap_df['date'].min()} to {evap_df['date'].max()}")
print(f"  Evap range: {evap_df['value'].min():.2f} to {evap_df['value'].max():.2f} mm/day")
print(f"  Evap mean: {evap_df['value'].mean():.2f} mm/day")

# Save to CSV
lat_str = f"{LATITUDE:.2f}"
lon_str = f"{LONGITUDE:.2f}"
model_scenario = f"{MODEL.replace(' ', '_')}_{SCENARIO}"
output_filename = f"{model_scenario}_{lat_str}_{lon_str}_evap.csv"
output_path = os.path.join(OUTPUT_DIR, output_filename)

evap_df.to_csv(output_path, index=False, encoding='utf-8', float_format='%.2f')
print(f"\n  [OK] Saved evaporation data to: {output_filename}")

print(f"\n{'='*70}")
print("[SUCCESS] EVAPORATION CALCULATION COMPLETED!")
print(f"{'='*70}")